# Polars 中文教程：北京昌平区空气质量数据分析

欢迎来到 Polars 中文教程！在这个案例中，我们将使用 Polars 分析北京昌平区的空气质量数据。

## 数据集介绍

我们将分析的数据来自北京市昌平区 2013 年至 2017 年的空气质量监测数据，包含了 PM2.5、PM10、SO2、NO2、CO、O3 等污染物浓度以及温度、湿度、风速等气象参数。

## 学习目标

通过本教程，您将学习到：
1. 如何使用 Polars 加载和探索数据
2. Polars 的基本数据操作
3. 数据清洗和预处理
4. 数据分组和聚合操作
5. 数据可视化
6. 高级分析技巧

In [ ]:
# 导入必要的库
import polars as pl
import matplotlib.pyplot as plt
import numpy as np

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print(f"Polars 版本: {pl.__version__}")

## 1. 数据加载与初步探索

首先，我们使用 Polars 加载数据并进行初步探索。

In [ ]:
# 定义数据类型以避免解析错误
dtypes = {
    "No": pl.Int64,
    "year": pl.Int64,
    "month": pl.Int64,
    "day": pl.Int64,
    "hour": pl.Int64,
    "PM2.5": pl.Float64,
    "PM10": pl.Float64,
    "SO2": pl.Float64,
    "NO2": pl.Float64,
    "CO": pl.Float64,
    "O3": pl.Float64,
    "TEMP": pl.Float64,
    "PRES": pl.Float64,
    "DEWP": pl.Float64,
    "RAIN": pl.Float64,
    "wd": pl.String,
    "WSPM": pl.Float64,
    "station": pl.String
}

# 使用 Polars 读取 CSV 数据
df = pl.read_csv(
    "PRSA_Data_Changping_20130301-20170228.csv", 
    schema_overrides=dtypes,
    null_values=["NA", "NaN", "null", ""]
)

print(f"数据加载成功！共有 {df.height} 行，{df.width} 列")
print("\n列名:", df.columns)

In [ ]:
# 查看前几行数据
print("前5行数据:")
df.head()

In [ ]:
# 查看数据类型
print("数据类型:")
for col, dtype in zip(df.columns, df.dtypes):
    print(f"  {col}: {dtype}")

In [ ]:
# 统计缺失值
print("缺失值统计:")
null_counts = df.null_count()
for col, count in zip(df.columns, null_counts.row(0)):
    if count > 0:
        print(f"  {col}: {count} 个缺失值")

## 2. 基本统计分析

现在让我们对数据进行一些基本的统计分析。

In [ ]:
# 计算数值型列的基本统计信息
numeric_cols = df.select(pl.col(pl.NUMERIC_DTYPES)).columns
print("数值型列的基本统计信息:")
df.select(numeric_cols).describe()

In [ ]:
# 计算各污染物的平均值
pollutants = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3"]
print("各污染物平均浓度:")
for pollutant in pollutants:
    if pollutant in df.columns:
        avg_value = df.select(pl.col(pollutant).mean()).item()
        print(f"  {pollutant}: {avg_value:.2f} μg/m³")

## 3. 时间序列分析

接下来，我们按时间维度分析空气质量的变化趋势。

In [ ]:
# 按年份统计平均 PM2.5
print("年度平均 PM2.5 浓度:")
yearly_pm25 = df.group_by("year").agg(
    pl.col("PM2.5").mean().alias("avg_PM2.5")
).sort("year")
yearly_pm25

In [ ]:
# 按月份统计平均 PM2.5
print("月度平均 PM2.5 浓度:")
monthly_pm25 = df.group_by("month").agg(
    pl.col("PM2.5").mean().alias("avg_PM2.5")
).sort("month")
monthly_pm25

In [ ]:
# 可视化年度趋势
plt.figure(figsize=(10, 6))
years = yearly_pm25["year"].to_list()
avg_pm25 = yearly_pm25["avg_PM2.5"].to_list()
plt.plot(years, avg_pm25, marker='o')
plt.title("PM2.5 年度平均浓度趋势")
plt.xlabel("年份")
plt.ylabel("PM2.5 (μg/m³)")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Polars 高级特性演示

现在让我们展示一些 Polars 的高级特性和强大功能。

In [ ]:
# 链式操作示例
print("链式操作示例 - 筛选2016年的数据并计算统计信息:")
result = (df
          .filter(pl.col("year") == 2016)
          .select([
              pl.col("PM2.5").mean().alias("mean_pm25"),
              pl.col("PM2.5").median().alias("median_pm25"),
              pl.col("PM2.5").max().alias("max_pm25"),
              pl.col("PM2.5").min().alias("min_pm25")
          ]))
result

In [ ]:
# 条件筛选示例
print("条件筛选示例 - 高污染天数统计 (PM2.5 > 100):")
high_pollution_days = df.filter(pl.col("PM2.5") > 100)
print(f"PM2.5超过100μg/m³的记录有 {high_pollution_days.height} 条")

In [ ]:
# 聚合操作示例
print("聚合操作示例 - 不同风向下的平均PM2.5:")
wind_pm25 = df.group_by("wd").agg(
    pl.col("PM2.5").mean().alias("avg_pm2.5")
).sort("avg_pm2.5", descending=True)
wind_pm25.head(10)

## 5. 相关性分析

最后，我们来分析不同污染物之间的相关性。

In [ ]:
# 计算污染物之间的相关性矩阵
pollutants = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3"]
corr_data = []

for p1 in pollutants:
    row = []
    for p2 in pollutants:
        corr_val = df.select(pl.corr(p1, p2)).item()
        row.append(corr_val)
    corr_data.append(row)

# 创建相关性DataFrame
corr_df = pl.DataFrame(corr_data, schema=pollutants)
corr_df.insert_at_idx(0, pl.Series(" ", pollutants))
print("污染物相关性矩阵:")
corr_df

## 总结

通过这个案例，我们展示了 Polars 在以下方面的强大功能：

1. **高效的数据加载**：Polars 能够快速加载大型 CSV 文件，并提供灵活的数据类型控制。
2. **直观的数据探索**：通过简单的方法就能获取数据的基本信息和统计摘要。
3. **强大的链式操作**：可以将多个数据操作连接在一起，使代码更加简洁易读。
4. **灵活的分组聚合**：提供了丰富的聚合函数和分组操作。
5. **高性能计算**：在处理大量数据时表现出色。

希望这个教程能帮助您更好地理解和使用 Polars 进行数据分析！